# Epoch vs Depth with CNOT

We were also interested to see if the qubit number and circuit depth had any impact on the effect of entanglement of the system. This code calculates the best test accuracy from each combination of 1-8 qubits and a depth of 1-10, 10 times, and outputs these files using pickle.dump. This version using the quantum processing unit with CNOT gates. The purpose of this code is to create data to be analysed using the code Analyse_Epoch_vs_Depth_Data. In this analysis, a heat map is created of the average best test accuracy for each combination of qubit number and depth. This is to analyse the effect of these two parameters on best test accuracy, and whether these parameters impact the results with and without the entangling CNOT gates. 


In [1]:
import time
import os
import copy
import pandas as pd
import random

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "1"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from Base import *

# Parallelizing using Pool.apply()
import multiprocessing as mp
# from pathos.multiprocessing import ProcessingPool as Pool
# # from pathos.pools import _ThreadPool
from multiprocessing.pool import ThreadPool as Pool



In [2]:
#setting main hyperparameters of the model 
             # Number of qubits
step = 0.0004               # Learning rate
batch_size = 4              # Number of samples for each training step
num_epochs = 15             # Number of training epochs
           # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()    # Start of the computation timer



#configure pytorch 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#Transform Data and define training and test data 
data_dir = data_dir = os.getcwd() + '/_data/hymenoptera_data_c'
data_transforms = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ] )

# Create Imagedata sets
image_datasets = datasets.ImageFolder(
       data_dir, data_transforms
    )

# Load data and get class names
dataset_size = len(image_datasets)
class_names = image_datasets.classes

In [4]:
n_qubits = np.arange(1, 8).tolist()
q_depth = np.arange(1, 8).tolist()

R=10

Nn=len(n_qubits)
Nq=len(q_depth)
B=np.zeros((Nn,Nq, R))
A=np.zeros((Nn,Nq, R))
Errors = np.zeros((Nn, Nq))
for i in range(Nn):
    for j in range(Nq):
        #Variational Quantum Circuit 

        #initialise pennylane
        dev = qml.device("default.qubit", wires=n_qubits[i])
        
        def H_layer(nqubits):
            """Layer of single-qubit Hadamard gates.
            """
            for idx in range(nqubits):
                qml.Hadamard(wires=idx)


        def RY_layer(w):
            """Layer of parametrized qubit rotations around the y axis.
            """
            for idx, element in enumerate(w):
                qml.RY(element, wires=idx)
                
        def entangling_layer(nqubits):
            """Layer of CNOTs followed by another shifted layer of CNOT.
            """
            # In other words it should apply something like :
            # CNOT  CNOT  CNOT  CNOT...  CNOT
            #   CNOT  CNOT  CNOT...  CNOT
            for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
                qml.CNOT(wires=[i, i + 1])
            for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
                qml.CNOT(wires=[i, i + 1])


        #Define Quantum Circuit 

        @qml.qnode(dev, interface="torch")
        def quantum_net(q_input_features, q_weights_flat):
            """
            The variational quantum circuit.
            """

            # Reshape weights
            q_weights = q_weights_flat.reshape(q_depth[j], n_qubits[i])

            # Start from state |+> , unbiased w.r.t. |0> and |1>
            H_layer(n_qubits[i])

            # Embed features in the quantum node
            RY_layer(q_input_features)

            # Sequence of trainable variational layers
            for k in range(q_depth[j]):
                entangling_layer(n_qubits[i])
                RY_layer(q_weights[k])

            # Expectation values in the Z basis
            exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits[i])]
            return tuple(exp_vals)


        #Dressed Quantum Circuit (primes data for QPU and to be extraced out the other end)
        class DressedQuantumNet(nn.Module):
            """
            Torch module implementing the *dressed* quantum net.
            """

            def __init__(self):
                """
                Definition of the *dressed* layout.
                """
        # use linear layer to reduce 512 to 4. 
                super().__init__()
                self.pre_net = nn.Linear(512, n_qubits[i])
                self.q_params = nn.Parameter(q_delta * torch.randn(q_depth[j] * n_qubits[i]))
                self.post_net = nn.Linear(n_qubits[i], 2)

            def forward(self, input_features):
                """
                Defining how tensors are supposed to move through the *dressed* quantum
                net.
                """

        #optimise features of linear reduction 
                # obtain the input features for the quantum circuit
                # by reducing the feature dimension from 512 to 4
                pre_out = self.pre_net(input_features)
                q_in = torch.tanh(pre_out) * np.pi / 2.0

                # Apply the quantum circuit to each element of the batch and append to q_out
                q_out = torch.Tensor(0, n_qubits[i])
                q_out = q_out.to(device)
                for elem in q_in:
                    q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
                    q_out = torch.cat((q_out, q_out_elem))

                # return the two-dimensional prediction from the postprocessing layer
                return self.post_net(q_out)

        #Hybrid Model 
        model_hybrid = torchvision.models.resnet18(pretrained=True)

        for param in model_hybrid.parameters():
            param.requires_grad = False


        # # Notice that model_hybrid.fc is the last layer of ResNet18
        model_hybrid.fc = DressedQuantumNet()

        # # Use CUDA or CPU according to the "device" object.
        # model_hybrid = model_hybrid.to(device)

        #Training and Results 
        #We use, as usual in classification problem, the cross-entropy which is directly available within torch.nn.
        criterion = nn.CrossEntropyLoss()

        #We also initialize the Adam optimizer which is called at each training step in order to update the weights of the model.
        optimizer_hybrid = optim.Adam(model_hybrid.fc.parameters(), lr=step)

        #We schedule to reduce the learning rate by a factor of gamma_lr_scheduler every 10 epochs.
        exp_lr_scheduler = lr_scheduler.StepLR(
            optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
        )

        #Training and Classification 

        def train_model(model, criterion, optimizer, scheduler, num_epochs, dataloaders):
            

            since = time.time()
            best_model_wts = copy.deepcopy(model.state_dict())
            best_acc = 0.0
            best_loss = 10000.0  # Large arbitrary number
            best_acc_train = 0.0
            best_loss_train = 10000.0  # Large arbitrary number
            print("Training started:")

            for epoch in range(num_epochs):

                if epoch+1 == num_epochs:
                    print("At final epoch, Retraining entire model once")
                    for param in model_hybrid.parameters():
                        param.requires_grad = True

                # Each epoch has a training and validation phase
                for phase in ["train", "validation"]:
                    if phase == "train":
                        # Set model to training mode
                        model.train()
                    else:
                        # Set model to evaluate mode
                        model.eval()
                    running_loss = 0.0
                    running_corrects = 0

                    # Iterate over data.
                    n_batches = dataset_sizes[phase] // batch_size
                    it = 0
                    for inputs, labels in dataloaders[phase]:
                        since_batch = time.time()
                        batch_size_ = len(inputs)
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        optimizer.zero_grad()

                        # Track/compute gradient and make an optimization step only when training
                        with torch.set_grad_enabled(phase == "train"):
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)
                            loss = criterion(outputs, labels)
                            if phase == "train":
                                loss.backward()
                                optimizer.step()

                        # Print iteration results
                        running_loss += loss.item() * batch_size_
                        batch_corrects = torch.sum(preds == labels.data).item()
                        running_corrects += batch_corrects
                        print(
                            "Phase: {} Epoch: {}/{} Iter: {}/{} Batch time: {:.4f}".format(
                                phase,
                                epoch + 1,
                                num_epochs,
                                it + 1,
                                n_batches + 1,
                                time.time() - since_batch,
                            ),
                            end="\r",
                            flush=True,
                        )
                        it += 1

                    # Print epoch results
                    print(running_corrects, dataset_sizes[phase])
                    epoch_loss = running_loss / dataset_sizes[phase]
                    epoch_acc = running_corrects / dataset_sizes[phase]
                    print(
                        "Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}        ".format(
                            "train" if phase == "train" else "validation  ",
                            epoch + 1,
                            num_epochs,
                            epoch_loss,
                            epoch_acc,
                        )
                    )

                    # Check if this is the best model wrt previous epochs
                    if phase == "validation" and epoch_acc > best_acc:
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
                    if phase == "validation" and epoch_loss < best_loss:
                        best_loss = epoch_loss
                    if phase == "train" and epoch_acc > best_acc_train:
                        best_acc_train = epoch_acc
                    if phase == "train" and epoch_loss < best_loss_train:
                        best_loss_train = epoch_loss

                    # Update learning rate
                    if phase == "train":
                        scheduler.step()

            # Print final results
            model.load_state_dict(best_model_wts)
            time_elapsed = time.time() - since
            print(
                "Training completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
            )
            print("Best test loss: {:.4f} | Best test accuracy: {:.4f}".format(best_loss, best_acc))

            
            return best_acc_train, best_acc


        #perform the training 
        er = 0
        for t in range(R):
            print('sample=', t, '...', i+1, j+1)
            
            
            # Load new images
            # Image_datasets contains a list of tuples with tensor objects and a one hot encoded class
            # Randomly shuffle data and create independent subsets
            print('######## ---------- Loading New Images ------------ ##########')
            percentage = 0.65
            train_size = int(percentage*dataset_size)
            inds = np.arange(0, dataset_size)
            train_samples = np.array(random.sample(range(0, dataset_size), train_size))
            val_samples = np.delete(inds, train_samples)

            train_data = torch.utils.data.Subset(image_datasets, [i for i in train_samples])
            val_data = torch.utils.data.Subset(image_datasets, [i for i in val_samples])
            dataset_sizes = {x: len(dataset) for x, dataset in zip(["train", "validation"], [train_data, val_data])}

            # Create data loaders for independent data
            dataloaders ={ 
                x: torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
                for x, dataset in zip(["train", "validation"], [train_data, val_data])
            }

            model_hybrid = torchvision.models.resnet18(pretrained=True)

            for param in model_hybrid.parameters():
                param.requires_grad = False

            # Notice that model_hybrid.fc is the last layer of ResNet18
            model_hybrid.fc = DressedQuantumNet()  

            #We use, as usual in classification problem, the cross-entropy which is directly available within torch.nn.
            criterion = nn.CrossEntropyLoss()

            #We also initialize the Adam optimizer which is called at each training step in order to update the weights of the model.
            optimizer_hybrid = optim.Adam(model_hybrid.fc.parameters(), lr=step)

            #We schedule to reduce the learning rate by a factor of gamma_lr_scheduler every 10 epochs.
            exp_lr_scheduler = lr_scheduler.StepLR(
                optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
            )
            
            B[i,j,t], A[i,j,t] = train_model(model_hybrid, criterion, optimizer_hybrid, exp_lr_scheduler, num_epochs, dataloaders)
            
            if A[i,j,t] < 0.6:
                er += 1
                print("###### ERROR detected ", er, " ######")
                Errors[i][j] += 1
            

sample= 0 ... 1 1
######## ---------- Loading New Images ------------ ##########
Training started:
131 258train Epoch: 1/15 Iter: 65/65 Batch time: 0.1130
Phase: train Epoch: 1/15 Loss: 0.6999 Acc: 0.5078        
73 139 validation Epoch: 1/15 Iter: 35/35 Batch time: 0.1172
Phase: validation   Epoch: 1/15 Loss: 0.6929 Acc: 0.5252        
131 258train Epoch: 2/15 Iter: 65/65 Batch time: 0.1054
Phase: train Epoch: 2/15 Loss: 0.6936 Acc: 0.5078        
73 139 validation Epoch: 2/15 Iter: 35/35 Batch time: 0.1141
Phase: validation   Epoch: 2/15 Loss: 0.6927 Acc: 0.5252        
131 258train Epoch: 3/15 Iter: 65/65 Batch time: 0.1050
Phase: train Epoch: 3/15 Loss: 0.6932 Acc: 0.5078        
73 139 validation Epoch: 3/15 Iter: 35/35 Batch time: 0.1104
Phase: validation   Epoch: 3/15 Loss: 0.6924 Acc: 0.5252        
131 258train Epoch: 4/15 Iter: 65/65 Batch time: 0.1075
Phase: train Epoch: 4/15 Loss: 0.6929 Acc: 0.5078        
73 139 validation Epoch: 4/15 Iter: 35/35 Batch time: 0.1042
Phase:

In [ ]:
import seaborn as sns
x_axis_labels = n_qubits
y_axis_labels = q_depth


ax = sns.heatmap(A, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="RdYlGn", cbar_kws={'label': 'Test Accuracy'}, annot=True, linewidth=0.5)
#plt.xlabel('Qubit Number')
#plt.xlabel('Depth')
plt.show()

In [5]:
B

tensor([[[0.87984496, 0.89922481, 0.89534884, 0.48062016, 0.87209302,
          0.50387597, 0.87209302, 0.52325581, 0.91085271, 0.89147287],
         [0.8875969 , 0.87596899, 0.87984496, 0.87209302, 0.88372093,
          0.51937984, 0.87209302, 0.86821705, 0.87209302, 0.52325581],
         [0.8875969 , 0.89922481, 0.50387597, 0.48062016, 0.89147287,
          0.86821705, 0.84883721, 0.87596899, 0.51162791, 0.87984496],
         [0.49612403, 0.89147287, 0.47286822, 0.87209302, 0.88372093,
          0.88372093, 0.8875969 , 0.89147287, 0.85271318, 0.46511628],
         [0.88372093, 0.8875969 , 0.5       , 0.52325581, 0.8875969 ,
          0.89147287, 0.89534884, 0.50387597, 0.90697674, 0.53875969],
         [0.90697674, 0.89534884, 0.88372093, 0.87596899, 0.89147287,
          0.87984496, 0.89147287, 0.89147287, 0.87596899, 0.86821705],
         [0.91085271, 0.90697674, 0.88372093, 0.91472868, 0.50775194,
          0.88372093, 0.87209302, 0.87209302, 0.47286822, 0.8875969 ]],

        [[0

In [9]:
import pickle as pkl

filename = 'val_WCNOT.pt'
fileobject = open(filename, 'wb')

pkl.dump(A.numpy(), fileobject)
fileobject.close()

In [10]:
A

tensor([[[0.97122302, 0.94244604, 0.94244604, 0.49640288, 0.97122302,
          0.5323741 , 0.89928058, 0.56834532, 0.98561151, 0.97122302],
         [0.94964029, 0.94244604, 0.92086331, 0.94964029, 0.96402878,
          0.50359712, 0.94244604, 0.94244604, 0.96402878, 0.49640288],
         [0.94964029, 0.94244604, 0.5323741 , 0.49640288, 0.94964029,
          0.94964029, 0.9352518 , 0.92086331, 0.51798561, 0.94964029],
         [0.4676259 , 0.94244604, 0.51079137, 0.94244604, 0.94964029,
          0.95683453, 0.94964029, 0.94244604, 0.96402878, 0.60431655],
         [0.96402878, 0.95683453, 0.46043165, 0.49640288, 0.97122302,
          0.95683453, 0.9352518 , 0.45323741, 0.92086331, 0.4676259 ],
         [0.92805755, 0.94964029, 0.9352518 , 0.94964029, 0.94964029,
          0.95683453, 0.94964029, 0.97122302, 0.92805755, 0.94964029],
         [0.95683453, 0.9352518 , 0.95683453, 0.94244604, 0.52517986,
          0.97841727, 0.94964029, 0.95683453, 0.58992806, 0.96402878]],

        [[0

In [11]:
B

tensor([[[0.87984496, 0.89922481, 0.89534884, 0.48062016, 0.87209302,
          0.50387597, 0.87209302, 0.52325581, 0.91085271, 0.89147287],
         [0.8875969 , 0.87596899, 0.87984496, 0.87209302, 0.88372093,
          0.51937984, 0.87209302, 0.86821705, 0.87209302, 0.52325581],
         [0.8875969 , 0.89922481, 0.50387597, 0.48062016, 0.89147287,
          0.86821705, 0.84883721, 0.87596899, 0.51162791, 0.87984496],
         [0.49612403, 0.89147287, 0.47286822, 0.87209302, 0.88372093,
          0.88372093, 0.8875969 , 0.89147287, 0.85271318, 0.46511628],
         [0.88372093, 0.8875969 , 0.5       , 0.52325581, 0.8875969 ,
          0.89147287, 0.89534884, 0.50387597, 0.90697674, 0.53875969],
         [0.90697674, 0.89534884, 0.88372093, 0.87596899, 0.89147287,
          0.87984496, 0.89147287, 0.89147287, 0.87596899, 0.86821705],
         [0.91085271, 0.90697674, 0.88372093, 0.91472868, 0.50775194,
          0.88372093, 0.87209302, 0.87209302, 0.47286822, 0.8875969 ]],

        [[0